In [1]:
import geopandas
import pandas as pd
import glob
import shapefile
from tqdm import tqdm
import numpy as np
import pickle
import xlrd3
import os
import glob

In [2]:
# read poi from shp files
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names = []
for f in tqdm(shp_files, ncols=200):
    x = geopandas.read_file(f, engine="pyogrio")
    places_names.append(x.name)
    
names = pd.concat(places_names, axis=0)
names.to_csv('tianjin.csv', index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:38<00:00,  1.66s/it]


In [3]:
names = pd.read_csv('tianjin.csv').name

In [ ]:
greenbelt = 'greenbelt'
greenbelt_list = pd.read_excel('/home/yangwenhao/local/project/weibo_filter/app/data/' + greenbelt + '.xls')

In [86]:
names.head(2)

0    元记电气焊补胎钣金
1       元汇津通卸载
Name: name, dtype: object

In [14]:
def apply_method(df, location):
    return df.str.contains(location)

In [18]:
text_csv = 'all_list/' + '%s.csv'%('list2_img')
for i, texts in enumerate(pd.read_csv(text_csv, chunksize=500000)):
    texts.apply(apply_method, axis=0, location="天津")

In [33]:
texts[12:13]

,text
12,"A Bite of Italy"" Four Hands Showcase at Four S..."


In [ ]:
# location_names = greenbelt_list.name # names
location_names = names
numofprint = 4

# 'first_2w'
# for lst in ['first_2w', 'noimg_6w']:
for lst in ['list2_img', 'list2_woimg']:
    
    text_csv = 'all_list/' + '%s.csv'%(lst)
    if not os.path.exists(text_csv):
        data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx')
        table = data.sheet_by_index(0)#.sheets()[0]          #通过索引顺序获取

        # get full-text tweets
        texts = []
        if lst == 'first_2w':
            test_idx = 5
        elif lst == 'list2_img':
            test_idx = 4
        elif lst == 'list2_woimg':
            test_idx = 6
        else:
            test_idx = 6

        for i in range(1, table.nrows):
            texts.append(table.row(i)[test_idx].value)

        texts = pd.DataFrame(texts, columns=['text'])
        texts.to_csv('%s.csv'%(lst), index=False)
    else:
        texts = pd.read_csv(text_csv)
    
    # for i, texts in enumerate(pd.read_csv(text_csv, chunksize=50000)):
    
    # match all palces for all lines
    places,sparse_matirx = [],[]
    for idx, i in tqdm(enumerate(location_names.unique())): #[297604:]
        try:
            result = texts.text.str.contains(i) # 12:26:26
            # result = texts.apply(apply_method, axis=0, location=i) # 17:11:27
            # result = texts.parallel_apply(apply_method, axis=0, location=i) # 76:42:44
            if result.to_numpy().sum() > 0:
                places.append(i)
                sparse_matirx.append(result.to_numpy())
        except Exception as e:
            continue
        
        if (idx+1) % 200000 == 0 or (idx+1) == len(location_names.unique()):
            print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
            with open('all_list/place2index_%s_%d.pickle' % (lst,idx), 'wb') as f:
                pickle.dump([places, sparse_matirx], f)
            
            places,sparse_matirx = [],[]

    # save match results with pickle
#     print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
#     # with open('all_list/place2index_%s_%d.pickle' % (lst,i), 'wb') as f:
#     #     pickle.dump([places, sparse_matirx], f)
#     with open('all_list/place2index_%s.pickle' % (lst), 'wb') as f:
#         pickle.dump([places, sparse_matirx], f)
        
#     # print the most places in all tweets
#     name2num = []
#     for i,p in enumerate(places):
#         name2num.append(sparse_matirx[i].sum())

#     name2num = np.array(name2num)
#     order_num = np.flip(np.argsort(name2num))
#     np_places = np.array(places)
#     for p, n in zip(np_places[order_num][:numofprint], name2num[order_num][:numofprint]):
#         print("{:14<}".format(p), ":", n)

In [ ]:
# save match results with pickle
for lst in ['list2_img', 'list2_woimg']:
    
    text_csv = 'all_list/' + '%s_*.csv'%(lst)
    text_csvs = glob.glob(text_csv)

    print("出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
    # with open('all_list/place2index_%s_%d.pickle' % (lst,i), 'wb') as f:
    #     pickle.dump([places, sparse_matirx], f)
    with open('all_list/  %s.pickle' % (lst), 'wb') as f:
        pickle.dump([places, sparse_matirx], f)

    # print the most places in all tweets
    name2num = []
    for i,p in enumerate(places):
        name2num.append(sparse_matirx[i].sum())

    name2num = np.array(name2num)
    order_num = np.flip(np.argsort(name2num))
    np_places = np.array(places)
    for p, n in zip(np_places[order_num][:numofprint], name2num[order_num][:numofprint]):
        print("{:14<}".format(p), ":", n)

In [2]:
def MatchProcess(tweet_csv, loc_csv, pickle_path):
    texts = pd.read_csv(tweet_csv)
    location_names = pd.read_csv(loc_csv).name
    places,sparse_matirx = [],[]
    
    for i in tqdm(location_names.unique(), ncols=100): #[297604:]
        try:
            result = texts.text.str.contains(i)
            if result.to_numpy().sum() > 0:
                places.append(i)
                sparse_matirx.append(result.to_numpy())
        except Exception as e:
            continue
        
    print(tweet_csv, "\n 出现的地点数: ", len(places), " 记录的列数:", len(sparse_matirx))
    with open(pickle_path, 'wb') as f:
        pickle.dump([places, sparse_matirx], f)

In [50]:
numofcsvs = 14

for lst in ['list2_img', 'list2_woimg']:
    
    text_csv = 'all_list/' + '%s_*.csv'%(lst)
    text_csvs = glob.glob(text_csv)
    
    if len(text_csvs) == 0:
        if os.path.exists('all_list/%s.csv'%(lst)):
            texts = pd.read_csv('all_list/%s.csv'%(lst), index_col=0)
        else:
            data = xlrd3.open_workbook('/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xlsx')
            table = data.sheet_by_index(0)#.sheets()[0]          #通过索引顺序获取

            # get full-text tweets
            texts = []
            if lst == 'first_2w':
                test_idx = 5
            elif lst == 'list2_img':
                test_idx = 4
            elif lst == 'list2_woimg':
                test_idx = 6
            else:
                test_idx = 6

            for i in range(1, table.nrows):
                texts.append(table.row(i)[test_idx].value)

            texts = pd.DataFrame(texts, columns=['text'])
            texts.to_csv('all_list/%s.csv'%(lst))
        
        texts_len = len(texts)
        csv_len = int(texts_len/numofcsvs)+1
        print('All lines: ', texts_len, 'Sub csv lines: ', csv_len)
        
        for i in range(numofcsvs):
            start = i * csv_len
            end = min((i+1) * csv_len, texts_len)
            if i+1 == numofcsvs:
                end = texts_len
                
            texts[start:end].to_csv('all_list/%s_%d.csv'%(lst, i))

All lines:  289992 Sub csv lines:  20714
All lines:  176968 Sub csv lines:  12641


In [3]:
text_csv = 'all_list/' + 'list2_*_*.csv'
text_csvs = glob.glob(text_csv)

In [ ]:
from multiprocessing import Pool

pool = Pool(processes=14)  # 创建nj个进程
loc_csv = 'tianjin.csv'

for tweet_csv in text_csvs:
    # MatchProcess(tweet_csv, loc_csv, pickle_path)
    pickle_path = tweet_csv.replace('csv', 'pickle')
    pool.apply_async(MatchProcess, args=(tweet_csv, loc_csv, pickle_path))
    
pool.close()  # 关闭进程池，表示不能在往进程池中添加进程
pool.join()  # 等待进程池中的所有进程执行完毕，必须在close()之后调用

In [40]:
texts = pd.read_csv('all_list/%s_%d.csv'%('list2_img', 1), index_col=0)

In [ ]:
# sparse_matirx.shape

In [4]:
# read poi and locations from shp files
shp_files = glob.glob('/home/yangwenhao/local/project/weibo_filter/app/data/tianjin/*.shp')

places_names_loc = []
for f in tqdm(shp_files):
    # print(f)
    x = geopandas.read_file(f, engine="pyogrio")
    places_names_loc.append(x[['name', 'geometry']])
    
places_names_loc = pd.concat(places_names_loc)
names2loc = {}
for l in places_names_loc.iterrows():
    # print(l[1].values[0], l[1].values[1],)
    names2loc[l[1].values[0]] = l[1].values[1]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:21<00:00,  1.05it/s]


In [32]:
print(x.geometry[0].x, x.geometry[0].y) 
# str(x.geometry[0])

116.7644123073674 38.82259674787439


'POINT (116.7644123073674 38.82259674787439)'

In [6]:
places_names_loc.head(2)

,name,geometry
0,元记电气焊补胎钣金,POINT (116.76441 38.82260)
1,元汇津通卸载,POINT (116.75977 38.82440)


In [ ]:
# read additional cols
with open('place2index.pickle', 'rb') as f:
    [places, sparse_matirx] = pickle.load(f)

places_np = np.array(places)
sparse_np = np.array(sparse_matirx) #.shape

append_cells = []
for l in range(sparse_np.shape[1]):
    row_cells = []
    for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
        i_xy = names2loc[i]
        row_cells.append([i, i_xy.x, i_xy.y])
    append_cells.append(row_cells)

In [ ]:
append_cells[:10]

[[['酒店', 117.70100359480168, 39.296045987523456],
  ['天津', 117.17989775825995, 39.12978780211583],
  ['天津瑞吉金融街酒店', 117.19148461137928, 39.12858938331741],
  ['金融街', 117.20638076565724, 39.12375796277156]],
 [['饭', 117.54906841621506, 39.41817775960832]],
 [['小白楼', 117.20932596977684, 39.1159842943953],
  ['天津', 117.17989775825995, 39.12978780211583],
  ['白楼', 117.28157823731766, 39.47219240381378],
  ['西洋', 117.5147029707013, 39.03815707550861]],
 [['雾造', 116.91964343118586, 38.936991545703805],
  ['津南区', 117.35020535412332, 38.93594862947365],
  ['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]],
 [['天津', 117.17989775825995, 39.12978780211583]]]

In [1]:
import xlrd
import xlwt
from xlutils.copy import copy

In [89]:
green_loc = set(location_names.to_list())

In [92]:
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
read_cols = [2, 3, 5, 6, 7, 8, 9, 10]

for lst in ['first_2w']: #, 
    # read additional cols
    with open('place2index.pickle', 'rb') as f:
        [places, sparse_matirx] = pickle.load(f)

    places_np = np.array(places)
    sparse_np = np.array(sparse_matirx) #.shape

    append_cells = []
    for l in range(sparse_np.shape[1]):
        row_cells = []
        for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
            i_xy = names2loc[i]
            row_cells.append([i, i_xy.x, i_xy.y])
        append_cells.append(row_cells)
    
    max_los = 1
    for i in append_cells:
        max_los = max(max_los, len(i))
    
    read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xls'
    save_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '_loc.xls'

    read_writebook = xlrd.open_workbook(filename=read_lst)
    # writebook = copy(wb=read_writebook)
    # writesheet = writebook.get_sheet(0)
    
    writebook = xlwt.Workbook(encoding='utf-8')
    # 添加sheet 写入excel, 参数对应 行, 列, 值
    poi_sheet     = writebook.add_sheet(u'地点检索到公园POI',   cell_overwrite_ok=True)
    
    npoi_sheet    = writebook.add_sheet(u'地点检索到非公园POI', cell_overwrite_ok=True)
    tianjin_sheet = writebook.add_sheet(u'地点仅检索到天津',    cell_overwrite_ok=True)
    
    poi_row, npoi_row, tianjin_row = 1, 1, 1
    
# 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
    for sheet in [poi_sheet, npoi_sheet, tianjin_sheet]:
        for i, h in enumerate(headers):
            sheet.write(0, i, label=h)
    
    for sheet in [poi_sheet, npoi_sheet]:
        for i in range(1, max_los+1):
            sheet.write(0, (i-1)*3+len(headers),   label='地点提取%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+1, label='经度%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+2, label='纬度%d'%i)
    
    tianjin_sheet.write(0, len(headers), label='地点提取')
        
    read_table = read_writebook.sheet_by_index(0)
    for i in range(1, nrows):
        this_line = read_table.row_slice(i)
        this_locs = append_cells[i-1]
        locs_names = [l[0] for l in this_locs]
        
        green_tweet = False
        for l in locs_names:
            if l in green_loc:
                green_tweet = True
                break
                
        if len(locs_names) == 1 and locs_names[0] == '天津':
            writesheet = tianjin_sheet
            this_row = tianjin_row
            tianjin_row += 1
        elif green_tweet:
            writesheet = poi_sheet
            this_row = poi_row
            poi_row += 1
        else:
            writesheet = npoi_sheet
            this_row = npoi_row
            npoi_row += 1
        
        writesheet.write(this_row, 0, label=len(append_cells[i-1]))
                         
        for j, c in enumerate(read_cols):
            writesheet.write(this_row, j+1, label=this_line[c].value)
        
        for j, (name, x, y) in enumerate(append_cells[i-1]):
            writesheet.write(this_row, j*3+len(headers),   label=name)
            writesheet.write(this_row, j*3+len(headers)+1, label=x)
            writesheet.write(this_row, j*3+len(headers)+2, label=y)

    writebook.save(save_lst)

In [96]:
headers = ["相关地点数量", "微博id", "微博主页", "文本", "日期", "图片", "转发数", "评论数", "点赞数"]
read_cols = [2, 3, 6, 7, 8, 9, 10, 11]

for lst in ['noimg_6w']: #'first_2w']: #, 
    # read additional cols
    with open('place2index_%s.pickle'%(lst), 'rb') as f:
        [places, sparse_matirx] = pickle.load(f)

    places_np = np.array(places)
    sparse_np = np.array(sparse_matirx) #.shape

    append_cells = []
    for l in range(sparse_np.shape[1]):
        row_cells = []
        for i in places_np[np.where(sparse_np[:, l] == True)[0]]:
            i_xy = names2loc[i]
            row_cells.append([i, i_xy.x, i_xy.y])
        append_cells.append(row_cells)
    
    max_los = 1
    for i in append_cells:
        max_los = max(max_los, len(i))
    
    read_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '.xls'
    save_lst = r'/home/yangwenhao/local/project/weibo_filter/app/data/' + lst + '_loc.xls'

    
    # writebook = copy(wb=read_writebook)
    # writesheet = writebook.get_sheet(0)
    
    writebook = xlwt.Workbook(encoding='utf-8')
    # 添加sheet 写入excel, 参数对应 行, 列, 值
    poi_sheet     = writebook.add_sheet(u'地点检索到公园POI',   cell_overwrite_ok=True)
    npoi_sheet    = writebook.add_sheet(u'地点检索到非公园POI', cell_overwrite_ok=True)
    tianjin_sheet = writebook.add_sheet(u'地点仅检索到天津',    cell_overwrite_ok=True)
    
    poi_row, npoi_row, tianjin_row = 1, 1, 1
    
# 相关地点数量	微博id	微博主页	文本	日期	图片	转发数	评论数	点赞数
    for sheet in [poi_sheet, npoi_sheet, tianjin_sheet]:
        for i, h in enumerate(headers):
            sheet.write(0, i, label=h)
    
    for sheet in [poi_sheet, npoi_sheet]:
        for i in range(1, max_los+1):
            sheet.write(0, (i-1)*3+len(headers),   label='地点提取%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+1, label='经度%d'%i)
            sheet.write(0, (i-1)*3+len(headers)+2, label='纬度%d'%i)
    
    tianjin_sheet.write(0, len(headers), label='地点提取')
    
    read_writebook = xlrd.open_workbook(filename=read_lst)
    read_table = read_writebook.sheet_by_index(0)
    nrows = read_table.nrows
    print("Num of rows: ", nrows)
    
    for i in range(1, nrows):
        this_line = read_table.row_slice(i)
        this_locs = append_cells[i-1]
        locs_names = [l[0] for l in this_locs]
        
        green_tweet = False
        for l in locs_names:
            if l in green_loc:
                green_tweet = True
                break
                
        if len(locs_names) == 1 and locs_names[0] == '天津':
            writesheet = tianjin_sheet
            this_row = tianjin_row
            tianjin_row += 1
        elif green_tweet:
            writesheet = poi_sheet
            this_row = poi_row
            poi_row += 1
        else:
            writesheet = npoi_sheet
            this_row = npoi_row
            npoi_row += 1
        
        writesheet.write(this_row, 0, label=len(append_cells[i-1]))
                                 
        for j, c in enumerate(read_cols):
            writesheet.write(this_row, j+1, label=this_line[c].value)
        
        for j, (name, x, y) in enumerate(append_cells[i-1]):
            writesheet.write(this_row, j*3+len(headers), label=name)
            writesheet.write(this_row, j*3+len(headers)+1, label=x)
            writesheet.write(this_row, j*3+len(headers)+2, label=y)

    writebook.save(save_lst)

Num of rows:  5540
